In [3]:
import json
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Document, VectorStoreIndex, Settings
from llama_index.vector_stores.pinecone import PineconeVectorStore
from pinecone import Pinecone
from tqdm.notebook import tqdm

In [ ]:
PINECONE_API_KEY = ""
PINECONE_ENVIRONMENT = "us-east-1"
HUGGINGFACE_API_KEY = ""

In [5]:
SINGLETON_FILE = "data/total_kor_counsel_bot.jsonl"
MULTITURN_FILE = "data/total_kor_multiturn_counsel_bot.jsonl"

In [14]:
def load_singleton_data(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            item = json.loads(line)
            data.append(Document(text=item['input'], metadata={"output": item['output']}))
    return data

In [28]:
def load_multiturn_data(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        conversation = []  # 대화를 저장할 리스트
        for line in f:
            try:
                items = json.loads(line)  # 한 줄을 JSON으로 파싱
                for item in items:
                    conversation.append(f"{item['speaker']}: {item['utterance']}")
                # 한 줄이 끝나면 대화를 하나의 문서로 저장
                input_text = ' '.join(conversation[:-1])  # 마지막 발화를 제외한 전체 대화
                output_text = conversation[-1]  # 마지막 발화 (상담사의 마지막 응답)
                data.append(Document(text=input_text, metadata={"output": output_text}))
                conversation = []  # 다음 대화를 위해 초기화
            except json.JSONDecodeError:
                print(f"Error decoding JSON: {line}")
            except KeyError as e:
                print(f"KeyError: {e} in line: {line}")
    return data


In [24]:
print("싱글턴 데이터 로딩 중...")
singleton_data = load_singleton_data(SINGLETON_FILE)
print(f"총 {len(singleton_data)}개의 싱글턴 문서 로드 완료")

싱글턴 데이터 로딩 중...
총 13234개의 싱글턴 문서 로드 완료


In [20]:
with open(MULTITURN_FILE, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        print(line.strip())
        if i == 5:  # 처음 5줄만 출력
            break

[{"speaker": "상담사", "utterance": "안녕하세요. 상담사입니다. 무엇이 불편하시나요?"}, {"speaker": "내담자", "utterance": "내가 약간 중2병 같은 걸 증상을 보이고 있어요."}, {"speaker": "상담사", "utterance": "중2병 증상이라니, 어떤 증상이신 건가요?"}, {"speaker": "내담자", "utterance": "그러니까 공부하기 싫어하고, 공격적이고, 좀 무례하게 말하고 싶은 게 많아져서 그런 거예요."}, {"speaker": "상담사", "utterance": "그런 증상이 있으니까 힘드시겠죠. 중2병 같은 것이라고 생각하시는 이유는 무엇인가요?"}, {"speaker": "내담자", "utterance": "막 공부 안하고 이것저것 들먹이고 하고 싶은 게 너무 많아서 그런 거 같아요."}, {"speaker": "상담사", "utterance": "그런 것도 어쩔 수 없이 찾아오는 시기가 있으니까 무리하지 않도록 해야겠죠. 대학교를 가면서 나아질 것 같았는데, 오히려 더 심해진 것 같다고 하셨죠. 그 원인이 무엇인가요?"}, {"speaker": "내담자", "utterance": "그걸 제가 잘 몰라서 그런 것 같아요. 그냥 더 심해졌다고 느꼈어요."}, {"speaker": "상담사", "utterance": "대학교 생활이 신나고 재밌으신 건 어떤 점이 있나요?"}, {"speaker": "내담자", "utterance": "학과가 정말 좋아서 즐겁게 수업을 듣고 있어요. 학우들도 좋고 괜찮은 친구들도 많이 만나서 그런 것 같아요."}, {"speaker": "상담사", "utterance": "즐거운 일도 많이 있으면서 고민거리도 있는 것 같군요. 가사나 소설을 쓰시면서 마음을 풀기도 하신다고 하셨는데, 언제부터 그 습관이 생겨난 건가요?"}, {"speaker": "내담자", "utterance": "좋은 질문이에요. 좀 자세히 말씀드릴게요. 학교에서 어려운 일이 

In [29]:
print("멀티턴 데이터 로딩 중...")
multiturn_data = load_multiturn_data(MULTITURN_FILE)
print(f"총 {len(multiturn_data)}개의 멀티턴 문서 로드 완료")

멀티턴 데이터 로딩 중...
총 8731개의 멀티턴 문서 로드 완료


In [30]:
all_documents = singleton_data + multiturn_data
print(f"총 {len(all_documents)}개의 문서 로드 완료")

총 21965개의 문서 로드 완료


In [31]:
pc = Pinecone(api_key=PINECONE_API_KEY)

In [33]:
index_name = "counselgpt-index"
dimension = 768  # 벡터 차원 (임베딩 모델에 따라 변경 필요)
metric = "cosine"

In [34]:
from pinecone import ServerlessSpec

In [35]:
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=dimension,
        metric=metric,
        spec=ServerlessSpec(
            cloud="aws",  # 사용할 클라우드 제공자 (예: "aws", "gcp")
            region="us-east-1"  # 사용할 리전
        )
    )

In [43]:
pinecone_index = pc.Index(index_name)

In [42]:
Settings.llm = HuggingFaceInferenceAPI(
    model_name="google/gemma-2-9b-it",
    token=HUGGINGFACE_API_KEY
)
Settings.embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/distiluse-base-multilingual-cased-v2",
    token=HUGGINGFACE_API_KEY
)

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

c:\Users\USER\anaconda3\envs\hi\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\AppData\Local\llama_index\models--sentence-transformers--distiluse-base-multilingual-cased-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.69k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

2_Dense%2Fconfig.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

In [46]:
Settings.chunk_size = 2048

In [44]:
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)

In [47]:
print("인덱스 생성 및 문서 추가 중...")
index = VectorStoreIndex.from_documents(
    all_documents,
    vector_store=vector_store,
    show_progress=True
)

print("인덱싱 완료!")

인덱스 생성 및 문서 추가 중...


Parsing nodes:   0%|          | 0/21965 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
query_engine = index.as_query_engine()